In [41]:
import os
from docx import Document
import csv
import spacy
import re
import pandas as pd

nlp = spacy.load("en_core_web_sm")
folder_path = "input_files/"

for filename in os.listdir(folder_path):
    if filename.endswith(".docx"):
        file_path = os.path.join(folder_path, filename)
        print(f"Reading file: {filename}")
        doc = Document(file_path)
        content = []
        for para in doc.paragraphs:
            content.append(para.text.split('Mom:')[-1].strip())
        # content='\n'.join(full_text)
        # print(content)
        content_editted=list(map(lambda x: x.split('Mom:')[-1].strip(),content))
        complete_sentence_list=[]
        # print(content_editted)
        for dialogue in content_editted:
            split_by_quotes = re.split(r'(\".*\")', dialogue)
            

            # Step 2: For each part, split by "and" and append to the final list
            final_split = []
            for part in split_by_quotes:
                # Step 2.1: Split by "and" for both quoted and non-quoted parts
                and_splits = re.split(r'\s+and\s+', part)
                
                # Step 2.2: Re-append 'and' to all but the last part in the split list
                for i, split in enumerate(and_splits):
                    if i >= 1:
                        final_split.append('and ' + split.strip())
                    else:
                        final_split.append(split.strip())

            # Filter out any empty strings
            final_split = [part for part in final_split if part]
            for split in final_split:


                doc = nlp(split)
                temp_sentence_list=[]
                for sent in doc.sents:
                    find_noun=1
                    start_index=0
                    end_index=-1
                    index_list=[]
                    index_len=0
                    for index,token in enumerate(sent):
                        # print(token,index)
                        if token.text==',' or token.text=='.':
                            if index_len!=0:
                                index_list[-1][1]=index+1
                                end_index=index+1
                                start_index=end_index
                            else:
                                index_list.append([start_index,index+1])
                                end_index=index+1
                                start_index=end_index
                        elif find_noun==1:
                            if token.pos_ in ["NOUN", "PROPN", "PRON"]:
                                # print(token.text)
                                end_index=index
                                index_list.append([start_index,end_index])
                                index_len+=1
                                start_index=index
                                find_noun=0
                        else:
                            if token.pos_ in ["VERB", "AUX"]:
                                # print(token.text)
                                find_noun=1
                    index_list.append([start_index,sent.end])
                    sentence_list=list(map(lambda x: str(sent[x[0]:x[1]]),index_list))
                    temp_sentence_list.extend(sentence_list)
                sentence_formed=' / '.join(temp_sentence_list)
                complete_sentence_list.append(sentence_formed)
        

        cleaned_texts = list(map(lambda text: re.sub(r'( / )+', ' / ', text), complete_sentence_list))

        cleaned_texts_strings=[]
        for dialogue in cleaned_texts:
            cleaned_texts_string=[]
            propositional_split=dialogue.split(' / ')
            new_split=[]
            temp_split=[]
            length_propositional_split=len(propositional_split)
            to_remove_indices=[]
            for part in propositional_split:
                if len(part)!=0:
                    if (len(part.split())==1 or len(part.split())==2) and part[-1]!='.' and part[0]!='"' and part[-1]!='"':
                        temp_split.append(part.encode().decode('unicode_escape').strip())
                    else:
                        if temp_split:
                            new_part = ' '.join(temp_split + [part])
                            new_split.append(new_part.encode().decode('unicode_escape').strip())
                            temp_split = []
                        else:
                            new_split.append(part.encode().decode('unicode_escape').strip())
            if temp_split:
                new_split += temp_split
            
            cleaned_texts_strings.extend(new_split)
                            


        print(cleaned_texts_strings)
        df_cleaned_texts_strings=pd.DataFrame(cleaned_texts_strings,columns=["PHRASES"])
        df_cleaned_texts_strings=df_cleaned_texts_strings.dropna()
        df_cleaned_texts_strings.to_csv('outputs/'+filename+'_output.tsv',sep="\t",quoting=csv.QUOTE_NONE,escapechar="\\",index=False)
        





Reading file: Copy of 6671-J Birth.docx
['Okay, birth story. So, because we\'re at even more high risk now with this diagnosis and my age and everything else, we were given regular visits. I had like a standing Friday visit with our OBGYN and at 37 weeks, at that particular visit, he said, "You\'re going to have a baby today."', 'Despite that fact that we should have been prepared, we weren\'t. Like I didn\'t even have a bag packed, crazy as it sounds. So, we-we live about a mile from the hospital, so I was like... He was saying, "Go to the hospital right now." And I\'m like, "Well, I have to go home and pack a bag." He\'s going, "No, you\'re going to the hospital right now."', 'The baby... one of the cords that goes to the baby was squeezed and so they weren\'t happy with the amount of blood flow and the size of the baby was 20% smaller and there\'s pressure from the amniotic fluid and, so they were at the point where they said, "He\'ll be safer outside the womb than inside, so go rig

In [16]:

df_final=pd.DataFrame(cleaned_texts_strings,columns=['segregated sentences'])
df_final['original sentences']=pd.DataFrame(content)
df_final.to_csv('8879-M Birth_segregated.csv',quoting=csv.QUOTE_NONNUMERIC, escapechar="\\")